### importing require packages

In [1]:
from __future__ import print_function

import json
import os
import numpy as np
import sys

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional

from nltk.tokenize import word_tokenize

Using TensorFlow backend.


### tokenizer function

In [2]:
# tokenizer: can change this as needed
# takes input one sentence at a time and returns individual words list
tokenize = lambda x: word_tokenize(x)

### create embeding and store weights for whole vocab

In [3]:
def create_embeddings(data_dir,
                      embeddings_path='lstm2-w2vec/embeddings.npz',
                      vocab_path='lstm2-w2vec/vocab.json',
                      **params):
    """
    Generate embeddings from a batch of text
    :param embeddings_path: where to save the embeddings
    :param vocab_path: where to save the word-index map
    """

    class SentenceGenerator(object):
        """
        Say we want to further preprocess the words from the files — convert to unicode, lowercase, 
        remove numbers, extract named entities… All of this can be done inside the MySentences iterator 
        and word2vec doesn’t need to know. All that is required is that the input yields one 
        sentence (list of utf8 words) after another
        
        """
        
        def __init__(self, dirname):
            self.dirname = os.path.join(os.getcwd(), dirname)
        
        ## iterator to yield tokenized format of one sentence at  a time
        def __iter__(self):
            for fname in os.listdir(self.dirname):
                for line in open(os.path.join(self.dirname, fname)):
                    yield tokenize(line.lower())

    """
    Gensim only requires that the input must provide sentences sequentially, when iterated over. 
    No need to keep everything in RAM: we can provide one sentence, process it, 
    forget it, load another sentence    
    """
    ## provide data directory and it will take sentences one at a time from all the files in directory
        
    sentences = SentenceGenerator(data_dir)
    model = Word2Vec(sentences, **params)
    
    weights = model.wv.syn0
    
    ## storing weights to be later used in keras embedding layer
    np.save(open(embeddings_path, 'wb'), weights)
    
    ## generating word2index
    vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    with open(vocab_path, 'w') as f:
        f.write(json.dumps(vocab))
    return model,weights

In [4]:
def load_vocab(vocab_path='lstm2-w2vec/vocab.json'):
    """
    Load word -> index and index -> word mappings
    :param vocab_path: where the word-index map is saved
    :return: word2idx, idx2word
    """

    with open(vocab_path, 'r') as f:
        data = json.loads(f.read())
    word2idx = data
    idx2word = dict([(v, k) for k, v in data.items()])
    return word2idx, idx2word

In [5]:
def word2vec_embedding_layer(embeddings_path='lstm2-w2vec/embeddings.npz'):
    """
    Generate an embedding layer word2vec embeddings
    :param embeddings_path: where the embeddings are saved (as a numpy file)
    :return: the generated embedding layer
    """
    
    weights = np.load(open(embeddings_path, 'rb'))
    layer = Embedding(input_dim=weights.shape[0],
                      output_dim=weights.shape[1],
                      #input_length=100,
                      weights=[weights])
    return layer

In [6]:
# specify embeddings in this environment variable
data_path = 'dumy_text'

# variable arguments are passed to gensim's word2vec model
w2v_model, weigths_ = create_embeddings(data_path, size=100, min_count=1,
                  window=5, sg=1, iter=25)

In [7]:
word2idx, idx2word = load_vocab()

In [8]:
vocab_size = len(word2idx.keys())
epoch = 1
batch_size = 32

In [9]:
filename = "/home/user/Documents/RNN/alice_edited.txt"
input_txt = open(filename).read()
input_txt = input_txt.lower()

In [10]:
tt = word_tokenize(input_txt)

In [11]:
x=[]
y=[]
window = 10
n_words = vocab_size
# generating dataset
for i in range(0, n_words - window, 1):
    seq_in = tt[i:i + window]
    seq_out = tt[i + window]
    x.append([word2idx[word] for word in seq_in])
    y.append(weigths_[word2idx[seq_out]])
n_patterns = len(x)
print ("Dataset size : ", n_patterns)

Dataset size :  3561


In [12]:
len(y[0])

100

In [13]:
y = np.array(y)

In [14]:
x = np.array(x)

In [33]:
##layer = Embedding(input_dim=weights.shape[0],output_dim=weights.shape[1],weights=[weights])
model = Sequential()
model.add(word2vec_embedding_layer())
model.add(Bidirectional(LSTM(1024,return_sequences=True)))
#model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(600)))
#model.add(Dropout(0.2))

model.add(Dense(100, activation='relu'))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         357100    
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 1024)        2510848   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 600)               3180000   
_________________________________________________________________
dense_2 (Dense)              (None, 100)               60100     
Total params: 6,108,048
Trainable params: 6,108,048
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.fit(x,y,batch_size=64,epochs=59,verbose=1,validation_split=0.2)

Train on 2848 samples, validate on 713 samples
Epoch 1/59
2848/2848 [==============================] - 51s - loss: 0.0411 - acc: 0.4041 - val_loss: 0.0587 - val_acc: 0.1122
Epoch 2/59
2848/2848 [==============================] - 51s - loss: 0.0407 - acc: 0.4105 - val_loss: 0.0583 - val_acc: 0.1164
Epoch 3/59
2848/2848 [==============================] - 51s - loss: 0.0401 - acc: 0.4442 - val_loss: 0.0592 - val_acc: 0.1206
Epoch 4/59
2848/2848 [==============================] - 51s - loss: 0.0394 - acc: 0.4417 - val_loss: 0.0585 - val_acc: 0.1234
Epoch 5/59
2848/2848 [==============================] - 51s - loss: 0.0388 - acc: 0.4835 - val_loss: 0.0602 - val_acc: 0.1192
Epoch 6/59
2848/2848 [==============================] - 51s - loss: 0.0383 - acc: 0.4853 - val_loss: 0.0598 - val_acc: 0.1136
Epoch 7/59
2848/2848 [==============================] - 56s - loss: 0.0376 - acc: 0.5158 - val_loss: 0.0601 - val_acc: 0.1248
Epoch 8/59
2848/2848 [==============================] - 54s - loss: 0.0

In [66]:
#np.array([x[2]]).shape

In [83]:
t1 = model.predict(np.array([x[79]]))

In [84]:
t1.shape

(1, 100)

In [85]:
w2v_model.similar_by_vector(t1[0])

[('could', 0.6972576379776001),
 ('fancy', 0.6012360453605652),
 ('venture', 0.5958577990531921),
 ('dare', 0.588391125202179),
 ('answer', 0.5864632725715637),
 ('remember', 0.5841941237449646),
 ('wondering', 0.5829741954803467),
 ('well', 0.5812236666679382),
 ('might', 0.568535327911377),
 ('eggs', 0.5633950233459473)]

In [78]:
start = 230
pattern = list(x[start])
print("\"",' '.join(idx2word[index] for index in pattern))
    
for i in range(10):
    prediction = model.predict(np.array([pattern]))
    #index = 0 #np.argmax(prediction)0
    pred_word = w2v_model.similar_by_vector(prediction[0])[0][0]
    sys.stdout.write(pred_word+" ")
    pattern.append(word2idx[pred_word])
    pattern = pattern[1:len(pattern)]


" hurried on , alice started to her feet , for
it knowledge , sister was sister across the run sister 